In [1]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#import numpy
# Load training data
from pyspark.ml.linalg import SparseVector
# from pyspark.python.pyspark.shell import spark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [2]:
data = spark.read.load("adult_data.csv", format="csv", header=True, delimiter=",")
data.show(5)

+---+------+---+---+----+---+---+
|  1|     3|  5| 10|  11| 12| 13|
+---+------+---+---+----+---+---+
| 39| 77516| 13|  1|2174|  0| 40|
| 50| 83311| 13|  1|   0|  0| 13|
| 38|215646|  9|  1|   0|  0| 40|
| 53|234721|  7|  1|   0|  0| 40|
| 28|338409| 13|  2|   0|  0| 40|
+---+------+---+---+----+---+---+
only showing top 5 rows



In [3]:
from pyspark.sql.types import *

# Change column type
data = data.withColumn("1", data["1"].cast(IntegerType()))
data = data.withColumn("3", data["3"].cast(IntegerType()))
data = data.withColumn("5", data["5"].cast(IntegerType()))
data = data.withColumn("10", data["10"].cast(IntegerType()))
data = data.withColumn("11", data["11"].cast(IntegerType()))
data = data.withColumn("12", data["12"].cast(IntegerType()))
data = data.withColumn("13", data["13"].cast(IntegerType()))

data.printSchema()

root
 |-- 1: integer (nullable = true)
 |-- 3: integer (nullable = true)
 |-- 5: integer (nullable = true)
 |-- 10: integer (nullable = true)
 |-- 11: integer (nullable = true)
 |-- 12: integer (nullable = true)
 |-- 13: integer (nullable = true)



In [4]:
data = data.withColumn("label", data['10'] - 0)
data.show(5)

+---+------+---+---+----+---+---+-----+
|  1|     3|  5| 10|  11| 12| 13|label|
+---+------+---+---+----+---+---+-----+
| 39| 77516| 13|  1|2174|  0| 40|    1|
| 50| 83311| 13|  1|   0|  0| 13|    1|
| 38|215646|  9|  1|   0|  0| 40|    1|
| 53|234721|  7|  1|   0|  0| 40|    1|
| 28|338409| 13|  2|   0|  0| 40|    2|
+---+------+---+---+----+---+---+-----+
only showing top 5 rows



In [5]:
assem = VectorAssembler(inputCols=data.columns[0:7], outputCol='features')
data = assem.transform(data)
data.show(5)

+---+------+---+---+----+---+---+-----+--------------------+
|  1|     3|  5| 10|  11| 12| 13|label|            features|
+---+------+---+---+----+---+---+-----+--------------------+
| 39| 77516| 13|  1|2174|  0| 40|    1|[39.0,77516.0,13....|
| 50| 83311| 13|  1|   0|  0| 13|    1|[50.0,83311.0,13....|
| 38|215646|  9|  1|   0|  0| 40|    1|[38.0,215646.0,9....|
| 53|234721|  7|  1|   0|  0| 40|    1|[53.0,234721.0,7....|
| 28|338409| 13|  2|   0|  0| 40|    2|[28.0,338409.0,13...|
+---+------+---+---+----+---+---+-----+--------------------+
only showing top 5 rows



In [6]:
# Split the data into train and test
train,test = data.randomSplit([0.6, 0.4], 1234)

In [7]:
# create the trainer and set its parameters
nb1 = NaiveBayes(labelCol="label", featuresCol="features", smoothing=1.0)

# train the model
model1 = nb1.fit(train)

# select example rows to display.
predictions = model1.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+---+-----+---+---+----+---+---+-----+--------------------+--------------------+--------------------+----------+
|  1|    3|  5| 10|  11| 12| 13|label|            features|       rawPrediction|         probability|prediction|
+---+-----+---+---+----+---+---+-----+--------------------+--------------------+--------------------+----------+
| 17|19752|  7|  2|   0|  0| 25|    2|[17.0,19752.0,7.0...|[-603.83024152897...|[3.55017080576790...|       1.0|
| 17|24090|  9|  2|   0|  0| 35|    2|[17.0,24090.0,9.0...|[-741.82504306404...|[1.30388265806149...|       1.0|
| 17|25051|  6|  1|   0|  0| 16|    1|[17.0,25051.0,6.0...|[-547.62221373539...|[8.55480582189268...|       1.0|
| 17|29571|  8|  1|   0|  0| 15|    1|[17.0,29571.0,8.0...|[-594.41135826275...|[4.22658921812652...|       1.0|
| 17|31007|  6|  2|   0|  0| 30|    2|[17.0,31007.0,6.0...|[-724.46084085428...|[6.78632746005661...|       1.0|
| 17|32607|  6|  1|   0|  0| 20|    1|[17.0,32607.0,6.0...|[-640.64253400357...|[1.1532817128748

In [8]:
# create the trainer and set its parameters
nb2 = NaiveBayes(labelCol="label", featuresCol="features", smoothing=10.0)

# train the model
model2 = nb2.fit(train)

# select example rows to display.
predictions = model2.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+---+-----+---+---+----+---+---+-----+--------------------+--------------------+--------------------+----------+
|  1|    3|  5| 10|  11| 12| 13|label|            features|       rawPrediction|         probability|prediction|
+---+-----+---+---+----+---+---+-----+--------------------+--------------------+--------------------+----------+
| 17|19752|  7|  2|   0|  0| 25|    2|[17.0,19752.0,7.0...|[-603.82833255900...|[3.54425060120333...|       1.0|
| 17|24090|  9|  2|   0|  0| 35|    2|[17.0,24090.0,9.0...|[-741.82292742258...|[1.30136607200351...|       1.0|
| 17|25051|  6|  1|   0|  0| 16|    1|[17.0,25051.0,6.0...|[-547.62132541304...|[8.54402615287241...|       1.0|
| 17|29571|  8|  1|   0|  0| 15|    1|[17.0,29571.0,8.0...|[-594.41044627901...|[4.22120922120825...|       1.0|
| 17|31007|  6|  2|   0|  0| 30|    2|[17.0,31007.0,6.0...|[-724.45916112928...|[6.77654354208634...|       1.0|
| 17|32607|  6|  1|   0|  0| 20|    1|[17.0,32607.0,6.0...|[-640.64174307001...|[1.1519318472885

In [9]:
# create the trainer and set its parameters
nb3 = NaiveBayes(labelCol="label", featuresCol="features", smoothing=100.0)

# train the model
model3 = nb3.fit(train)

# select example rows to display.
predictions = model3.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+---+-----+---+---+----+---+---+-----+--------------------+--------------------+--------------------+----------+
|  1|    3|  5| 10|  11| 12| 13|label|            features|       rawPrediction|         probability|prediction|
+---+-----+---+---+----+---+---+-----+--------------------+--------------------+--------------------+----------+
| 17|19752|  7|  2|   0|  0| 25|    2|[17.0,19752.0,7.0...|[-603.80927695553...|[3.48590625680393...|       1.0|
| 17|24090|  9|  2|   0|  0| 35|    2|[17.0,24090.0,9.0...|[-741.80180576775...|[1.27658500467266...|       1.0|
| 17|25051|  6|  1|   0|  0| 16|    1|[17.0,25051.0,6.0...|[-547.61244971568...|[8.4377234571296E...|       1.0|
| 17|29571|  8|  1|   0|  0| 15|    1|[17.0,29571.0,8.0...|[-594.40133447052...|[4.16816133976383...|       1.0|
| 17|31007|  6|  2|   0|  0| 30|    2|[17.0,31007.0,6.0...|[-724.44239778958...|[6.68008191248783...|       1.0|
| 17|32607|  6|  1|   0|  0| 20|    1|[17.0,32607.0,6.0...|[-640.63384131883...|[1.1386212643086

In [10]:
# create the trainer and set its parameters
nb4 = NaiveBayes(labelCol="label", featuresCol="features", modelType="multinomial")

# train the model
model4 = nb4.fit(train)

# select example rows to display.
predictions = model4.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+---+-----+---+---+----+---+---+-----+--------------------+--------------------+--------------------+----------+
|  1|    3|  5| 10|  11| 12| 13|label|            features|       rawPrediction|         probability|prediction|
+---+-----+---+---+----+---+---+-----+--------------------+--------------------+--------------------+----------+
| 17|19752|  7|  2|   0|  0| 25|    2|[17.0,19752.0,7.0...|[-603.83024152897...|[3.55017080576790...|       1.0|
| 17|24090|  9|  2|   0|  0| 35|    2|[17.0,24090.0,9.0...|[-741.82504306404...|[1.30388265806149...|       1.0|
| 17|25051|  6|  1|   0|  0| 16|    1|[17.0,25051.0,6.0...|[-547.62221373539...|[8.55480582189268...|       1.0|
| 17|29571|  8|  1|   0|  0| 15|    1|[17.0,29571.0,8.0...|[-594.41135826275...|[4.22658921812652...|       1.0|
| 17|31007|  6|  2|   0|  0| 30|    2|[17.0,31007.0,6.0...|[-724.46084085428...|[6.78632746005661...|       1.0|
| 17|32607|  6|  1|   0|  0| 20|    1|[17.0,32607.0,6.0...|[-640.64253400357...|[1.1532817128748

In [11]:
from pyspark.ml.classification import DecisionTreeClassifier

# create the trainer and set its parameters
nb3 = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth= 2)

# train the model
model3 = nb3.fit(train)

# select example rows to display.
predictions = model3.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+---+-----+---+---+----+---+---+-----+--------------------+-----------------+-------------+----------+
|  1|    3|  5| 10|  11| 12| 13|label|            features|    rawPrediction|  probability|prediction|
+---+-----+---+---+----+---+---+-----+--------------------+-----------------+-------------+----------+
| 17|19752|  7|  2|   0|  0| 25|    2|[17.0,19752.0,7.0...| [0.0,0.0,6409.0]|[0.0,0.0,1.0]|       2.0|
| 17|24090|  9|  2|   0|  0| 35|    2|[17.0,24090.0,9.0...| [0.0,0.0,6409.0]|[0.0,0.0,1.0]|       2.0|
| 17|25051|  6|  1|   0|  0| 16|    1|[17.0,25051.0,6.0...|[0.0,13008.0,0.0]|[0.0,1.0,0.0]|       1.0|
| 17|29571|  8|  1|   0|  0| 15|    1|[17.0,29571.0,8.0...|[0.0,13008.0,0.0]|[0.0,1.0,0.0]|       1.0|
| 17|31007|  6|  2|   0|  0| 30|    2|[17.0,31007.0,6.0...| [0.0,0.0,6409.0]|[0.0,0.0,1.0]|       2.0|
| 17|32607|  6|  1|   0|  0| 20|    1|[17.0,32607.0,6.0...|[0.0,13008.0,0.0]|[0.0,1.0,0.0]|       1.0|
| 17|34019|  6|  1|   0|  0| 20|    1|[17.0,34019.0,6.0...|[0.0,13008.0,0

In [12]:
from pyspark.ml.classification import DecisionTreeClassifier

# create the trainer and set its parameters
nb3 = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth= 10)

# train the model
model3 = nb3.fit(train)

# select example rows to display.
predictions = model3.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+---+-----+---+---+----+---+---+-----+--------------------+-----------------+-------------+----------+
|  1|    3|  5| 10|  11| 12| 13|label|            features|    rawPrediction|  probability|prediction|
+---+-----+---+---+----+---+---+-----+--------------------+-----------------+-------------+----------+
| 17|19752|  7|  2|   0|  0| 25|    2|[17.0,19752.0,7.0...| [0.0,0.0,6409.0]|[0.0,0.0,1.0]|       2.0|
| 17|24090|  9|  2|   0|  0| 35|    2|[17.0,24090.0,9.0...| [0.0,0.0,6409.0]|[0.0,0.0,1.0]|       2.0|
| 17|25051|  6|  1|   0|  0| 16|    1|[17.0,25051.0,6.0...|[0.0,13008.0,0.0]|[0.0,1.0,0.0]|       1.0|
| 17|29571|  8|  1|   0|  0| 15|    1|[17.0,29571.0,8.0...|[0.0,13008.0,0.0]|[0.0,1.0,0.0]|       1.0|
| 17|31007|  6|  2|   0|  0| 30|    2|[17.0,31007.0,6.0...| [0.0,0.0,6409.0]|[0.0,0.0,1.0]|       2.0|
| 17|32607|  6|  1|   0|  0| 20|    1|[17.0,32607.0,6.0...|[0.0,13008.0,0.0]|[0.0,1.0,0.0]|       1.0|
| 17|34019|  6|  1|   0|  0| 20|    1|[17.0,34019.0,6.0...|[0.0,13008.0,0

In [13]:
from pyspark.ml.classification import DecisionTreeClassifier

# create the trainer and set its parameters
nb3 = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth= 30)

# train the model
model3 = nb3.fit(train)

# select example rows to display.
predictions = model3.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+---+-----+---+---+----+---+---+-----+--------------------+-----------------+-------------+----------+
|  1|    3|  5| 10|  11| 12| 13|label|            features|    rawPrediction|  probability|prediction|
+---+-----+---+---+----+---+---+-----+--------------------+-----------------+-------------+----------+
| 17|19752|  7|  2|   0|  0| 25|    2|[17.0,19752.0,7.0...| [0.0,0.0,6409.0]|[0.0,0.0,1.0]|       2.0|
| 17|24090|  9|  2|   0|  0| 35|    2|[17.0,24090.0,9.0...| [0.0,0.0,6409.0]|[0.0,0.0,1.0]|       2.0|
| 17|25051|  6|  1|   0|  0| 16|    1|[17.0,25051.0,6.0...|[0.0,13008.0,0.0]|[0.0,1.0,0.0]|       1.0|
| 17|29571|  8|  1|   0|  0| 15|    1|[17.0,29571.0,8.0...|[0.0,13008.0,0.0]|[0.0,1.0,0.0]|       1.0|
| 17|31007|  6|  2|   0|  0| 30|    2|[17.0,31007.0,6.0...| [0.0,0.0,6409.0]|[0.0,0.0,1.0]|       2.0|
| 17|32607|  6|  1|   0|  0| 20|    1|[17.0,32607.0,6.0...|[0.0,13008.0,0.0]|[0.0,1.0,0.0]|       1.0|
| 17|34019|  6|  1|   0|  0| 20|    1|[17.0,34019.0,6.0...|[0.0,13008.0,0

In [14]:
from pyspark.ml.classification import DecisionTreeClassifier

# create the trainer and set its parameters
nb3 = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth= 100)

# train the model
model3 = nb3.fit(train)

# select example rows to display.
predictions = model3.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

IllegalArgumentException: 'requirement failed: DecisionTree currently only supports maxDepth <= 30, but was given maxDepth = 100.'

In [15]:
from pyspark.ml.classification import RandomForestClassifier

# create the trainer and set its parameters
nb3 = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=1)

# train the model
model3 = nb3.fit(train)

# select example rows to display.
predictions = model3.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+---+-----+---+---+----+---+---+-----+--------------------+-------------+-------------+----------+
|  1|    3|  5| 10|  11| 12| 13|label|            features|rawPrediction|  probability|prediction|
+---+-----+---+---+----+---+---+-----+--------------------+-------------+-------------+----------+
| 17|19752|  7|  2|   0|  0| 25|    2|[17.0,19752.0,7.0...|[0.0,0.0,1.0]|[0.0,0.0,1.0]|       2.0|
| 17|24090|  9|  2|   0|  0| 35|    2|[17.0,24090.0,9.0...|[0.0,0.0,1.0]|[0.0,0.0,1.0]|       2.0|
| 17|25051|  6|  1|   0|  0| 16|    1|[17.0,25051.0,6.0...|[0.0,1.0,0.0]|[0.0,1.0,0.0]|       1.0|
| 17|29571|  8|  1|   0|  0| 15|    1|[17.0,29571.0,8.0...|[0.0,1.0,0.0]|[0.0,1.0,0.0]|       1.0|
| 17|31007|  6|  2|   0|  0| 30|    2|[17.0,31007.0,6.0...|[0.0,0.0,1.0]|[0.0,0.0,1.0]|       2.0|
| 17|32607|  6|  1|   0|  0| 20|    1|[17.0,32607.0,6.0...|[0.0,1.0,0.0]|[0.0,1.0,0.0]|       1.0|
| 17|34019|  6|  1|   0|  0| 20|    1|[17.0,34019.0,6.0...|[0.0,1.0,0.0]|[0.0,1.0,0.0]|       1.0|
| 17|47199

In [16]:
from pyspark.ml.classification import RandomForestClassifier

# create the trainer and set its parameters
nb3 = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

# train the model
model3 = nb3.fit(train)

# select example rows to display.
predictions = model3.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+---+-----+---+---+----+---+---+-----+--------------------+--------------------+--------------------+----------+
|  1|    3|  5| 10|  11| 12| 13|label|            features|       rawPrediction|         probability|prediction|
+---+-----+---+---+----+---+---+-----+--------------------+--------------------+--------------------+----------+
| 17|19752|  7|  2|   0|  0| 25|    2|[17.0,19752.0,7.0...|[0.0,0.4258409785...|[0.0,0.0425840978...|       2.0|
| 17|24090|  9|  2|   0|  0| 35|    2|[17.0,24090.0,9.0...|[0.0,0.8885377616...|[0.0,0.0888537761...|       2.0|
| 17|25051|  6|  1|   0|  0| 16|    1|[17.0,25051.0,6.0...|[0.0,9.4258409785...|[0.0,0.9425840978...|       1.0|
| 17|29571|  8|  1|   0|  0| 15|    1|[17.0,29571.0,8.0...|[0.0,9.4258409785...|[0.0,0.9425840978...|       1.0|
| 17|31007|  6|  2|   0|  0| 30|    2|[17.0,31007.0,6.0...|[0.0,0.4258409785...|[0.0,0.0425840978...|       2.0|
| 17|32607|  6|  1|   0|  0| 20|    1|[17.0,32607.0,6.0...|[0.0,9.4258409785...|[0.0,0.942584097

In [17]:
from pyspark.ml.classification import RandomForestClassifier

# create the trainer and set its parameters
nb3 = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100)

# train the model
model3 = nb3.fit(train)

# select example rows to display.
predictions = model3.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+---+-----+---+---+----+---+---+-----+--------------------+--------------------+--------------------+----------+
|  1|    3|  5| 10|  11| 12| 13|label|            features|       rawPrediction|         probability|prediction|
+---+-----+---+---+----+---+---+-----+--------------------+--------------------+--------------------+----------+
| 17|19752|  7|  2|   0|  0| 25|    2|[17.0,19752.0,7.0...|[0.0,2.1437195591...|[0.0,0.0214371955...|       2.0|
| 17|24090|  9|  2|   0|  0| 35|    2|[17.0,24090.0,9.0...|[0.0,1.3459509418...|[0.0,0.0134595094...|       2.0|
| 17|25051|  6|  1|   0|  0| 16|    1|[17.0,25051.0,6.0...|[0.0,98.143719559...|[0.0,0.9814371955...|       1.0|
| 17|29571|  8|  1|   0|  0| 15|    1|[17.0,29571.0,8.0...|[0.0,98.580118776...|[0.0,0.9858011877...|       1.0|
| 17|31007|  6|  2|   0|  0| 30|    2|[17.0,31007.0,6.0...|[0.0,2.1437195591...|[0.0,0.0214371955...|       2.0|
| 17|32607|  6|  1|   0|  0| 20|    1|[17.0,32607.0,6.0...|[0.0,98.143719559...|[0.0,0.981437195